
### Load Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Define LLM and Embeddings

In [2]:
from langchain_openai import OpenAIEmbeddings

base_embeddings = OpenAIEmbeddings()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=1)

### Split Documents

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/util.txt")
documents = loader.load()

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='.', chunk_size=1200, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

### Define Chroma Vector DB

In [4]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs, 
                           base_embeddings, 
                           collection_metadata={'hnsw:space': 'cosine'})

### Instantiate *improved* HypotheticalDocumentEmbedder

In [5]:
from src.auto_hyde import HypotheticalDocumentEmbedder

hyde_embedder = HypotheticalDocumentEmbedder(
    llm_chain=llm, base_embeddings=base_embeddings
)

### Define Custom HyDE params

In [6]:
hypo_params = {
    'baseline_k': 20,
    'exploration_multiplier': 5,
    'verbose': True
}

### Get Embedding

In [7]:
query = 'what is the relationship between justice and happiness?'

hyde_embedding = hyde_embedder.embed_query(text=query, db=db, hypo_params=hypo_params)


>>> Extracting Keywords from your Query...
>>> ...Keywords Extracted: ['relationship', 'justice', 'happiness']


>>> Performing Initial Retrieval of 100 documents...


>>> Checking 80 Docs ranked after 20 for presence of keyword...
>>> ...69 neglected Docs identified


>>> Clustering neglected Docs...
>>> ...6 Clusters identified


>>> Generating Hypothetical Documents for each Doc Cluster...


### Hypo Doc 1 ###
In the quest to fathom the interrelation of justice and happiness, one might ponder their concurrence in the course of human affairs. Justice, as a societal construct, upholds a framework within which the individual and the collective aim to coexist harmoniously. Happiness, on the other hand, often springs from the well of contentment, shaped by the assurance of fairness and rectitude in one's surroundings. It follows, then, that a just society, by nurturing the equilibrium of rights and duties, creates fertile ground for happiness to flourish.

  Justice, in its purest form,


>>> Combining embeddings for hypothetical documents...


>>> Auto Hyde Embedding Complete!



In [8]:
hyde_embedding[:10]

[0.015876703333653572,
 -0.013635452586265312,
 0.021941788843565697,
 -0.02570370381768275,
 -0.015861831315729033,
 -0.0003427757204382006,
 0.0027591148428962454,
 -0.00883308151544041,
 -0.014893267477206646,
 -0.020748802766928837]